In this exercise we step-wise solve a vehicle routing problem (VRP) by separating it in multiple traveling salesman problems (TSP). To do so, we first cluster all locations. Second, we model the TSPs as an integer-linear program (ILP) and solve it using a freely available solver. We operate in the euclidean plane.

Let's start by importing a couple of useful libraries, needed in this exercise. Probably, those imports lead to a couple of problems, as pyomo has not been installed, yet. Pyomo is a modelling language Pyomo we will use to modle the TSP. Together with pyomo, we will use the freely available solver glpk. To use the solver glpk with Pyomo, it must be installed so the command "glpsol --help" can be given on the command line from any directory. 
- pip install pyomo 
- sudo apt install libglpk-dev python3.8-dev libgmp3-dev
- pip install glpk
- sudo apt install glpk-utils
- pip install -U scikit-learn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
import pyomo.environ as pyEnv

Further we need to define the input to our problem. First, we set the locations to visit, note that location 0 is the depot. Second, we define the number of clusters we want to divide them into, setting the number of TSPs. Further we define a set of colours to use for our plotting. Feel free to play with these parameters, once you solved the original assignment.

In [ ]:
locations= np.array([
(250, 250), # location 0 - the depot
(230, 0), # location 1
(449, 120), # location 2
(-1, 200), # location 3
(119, 105), # location 4
(372, 156), # location 5
(29, 355), # location 6
(338, 141), # location 7
(381, 444), # location 8
(172, 398), # location 9
(8, 401), # location 10
(111, 477), # location 11
(325, 484), # location 12
(332, 397), # location 13
(461, 483), # location 14
(-4, 36), # location 15
(299, 100), # location 16
(44, 40), # location 17
(122, 344), # location 18
(1, 8), # location 19
(225, 39)]) # location 20

n_clusters = 4
own_colors = ["purple", "green", "blue", "yellow", "black"]

#### Plot the starting situation

Start by plotting the depot location and the locations to visit. Tip: Use a scatter plot of matplotlib.

In [ ]:
plt.figure(figsize=(8,5))
# First, plot the depot only
plt.scatter(............, color="red")
# plot the customers locations
for..............: 
    plt.scatter(............, color="blue")

# Clustering

Here we divide all customer locations into groups (clusters). We do so based on their spatial relationships.<br>

#### Cluster all customer locations using K means of sklearn

K means clustering algorithm steps:
   - Choose a number of centroids (number of clusters).
   - Choose the same number of random points on the 2D canvas as centroids.
   - Calculate the distance of each data point from the centroids.
   - Allocate the data point to a cluster where its distance from the centroid is minimum.
   - Recalculate the new centroids.
   - Recalculate the distance from each data point to new centroids.
   - Repeat the steps from point 3, till no data point change its cluster.

To do so we leverage KMeans of sklearn.cluster, imported earlier. Note, there is no need to implement k means from scratch, only parameters are missing.

In [ ]:
kmean=KMeans(............)
kmean.fit(............) # pay attention to exclude the depot

The solution of the clustering can be accesed as follows:

In [ ]:
kmean.labels_

#### Plot the clusters

Repeat the previous plot and color each location the according to their cluster. Be aware that the clustering does not contain the depot.

In [ ]:
plt.figure(figsize=(8,5))
............  # Here multiple lines of code need to be filled

# Solving the TSP

#### Develop a function generating a cost matrix for a cluster

We define the cost of two locations as the distance between them. As we operate in the euclidian plane, the bee line can be used. For one cluster (TSP), including the depot, we store all costs in a so called cost matrix. 

In [ ]:
def distance(loc1, loc2):
    return ............

def def_cost_matrix(locations):
    cost_matrix = np.zeros((len(locations), len(locations))) # creates matrix with all values zero
    ............  # Here multiple lines of code need to be filled
    return cost_matrix

### Model the TSP

To model the TSP we use pyomo. A great source of information is http://www.opl.ufc.br/post/tsp/. We will define functions for the objective function and each constraint. Those will be utilised within the creation of the model, which we wrap into one function. We do so, such that we get a function "solve_tsp(locations_tsp, cost_matrix)", taking the locations and an associated cost matrix and returns the solution.

#### Define functions to model the objective function and the needed constraints

In [ ]:
def obj_func(model):
    return sum(model.x[i,j] * model.c[i,j] for ............)

def rule_const1(model,M):
    return sum(model.x[i,M] for i in model.N if i!=M ) == ............

def rule_const2(model,N):
    return sum(model.x[N,j] for j in model.M if j!=N) == ............

def rule_const3(model,i,j):
    if i!=j: 
        return ............
    else:
        #Please note that this else doesn't really say anything
        return model.u[i] - model.u[i] == 0 

#### Put everything together into one function creating a specific TSP and solving it

The model needs to be created first. To solve it we will call a solver (glpk). Last, the output of the solver needs to be shaped (to further work with it) and returned.

In [ ]:
def solve_tsp(locations_tsp, cost_matrix): # based on "http://www.opl.ufc.br/post/tsp/"
    #Model
    model = pyEnv.ConcreteModel()

    #Indexes for the cities
    model.M = pyEnv.RangeSet(len(locations_tsp))                
    model.N = pyEnv.RangeSet(len(locations_tsp))

    #Index for the dummy variable u
    model.U = pyEnv.RangeSet(2,len(locations_tsp))
    
    #Decision variables xij
    model.x = pyEnv.Var(............, within=pyEnv.Binary)

    #Dummy variable ui
    model.u = pyEnv.Var(model.N, within=pyEnv.NonNegativeIntegers,bounds=(0,len(locations_tsp)-1))
    
    #Cost Matrix cij
    model.c = pyEnv.Param(model.N, model.M,initialize=lambda model, i, j: ............)

    model.objective = pyEnv.Objective(rule=obj_func,sense=pyEnv.minimize)

    model.const1 = pyEnv.Constraint(model.M,rule=rule_const1)

    model.rest2 = pyEnv.Constraint(model.N,rule=rule_const2)

    model.rest3 = pyEnv.Constraint(model.U,model.N,rule=rule_const3)

    #model.print()

    #Solves
    solver = pyEnv.SolverFactory('glpk')
    result = solver.solve(model,tee = False)

    #Prints the results
    #print(result)

    l = list(model.x.keys())
    sol=[]
    for i in ............:
        if model.x[i]() != ............:
            if model.x[i]() != None:
                sol.append(i)
    #print(sol)
    
    #sort the solution
    sorted_sol = ............ #Initalize a list of visited location ids, always starting at the depot
    for i in sol:
        for ii in sol:
            last_loc = ............
            if ii[0] == ............:
                if ii[1] == 1: # stop if we are back at the depot
                    break
                else:
                    sorted_sol.append(ii[1])
    sorted_sol.append(............) # we go back to the depot
            
    return sorted_sol

#### Use the created functions to create and solve a TSP for each cluster

Here we apply the created function to each cluster of customer locations.

In [ ]:
#Solve TSP for each cluster
results = []
cities_cluster_save = [] 
for cluster_id in range(0, ............): # Solve each cluster separately
    print("Cluster no: {}".format(cluster_id))

    cities_cluster = [] # saved such that the used indices can be used later on
    cities_cluster.append(locations[0]) # depot is part of all clusters
    for loc in locations[1:][............ == ............]:
        cities_cluster.append(loc)

    cost_matrix = ............
    sol= ............
    print("Vehicle follows the route: {}".format(sol))
    
    #save solution
    results.append(sol)
    cities_cluster_save.append(cities_cluster)
    
print(results)

#### Visualise your results

Reuse the previous plot and add the tour given by solving the TSP for each cluster. Therefore connect to consecutively visited locations by a line using "plt.plot". Please, add your name and student number to the plot.

In [ ]:
plt.figure(figsize=(8,5))
............ # Here multiple lines of code need to be filled


plt.text(335,300, "Name")
plt.text(335,250, "000001")